<a href="https://colab.research.google.com/github/creatino/geocoding/blob/main/Reverse_geocoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install reverse_geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for reverse_geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268068 sha256=469b892f709c3845c4c983557100801e4e6c6e28b12d57d72e1c2ac841c117e1
  Stored in directory: /root/.cache/pip/wheels/17/3c/41/2bc89719586c2a5c53e9a527daa76a968a1288315c1ae2d904
Successfully built reverse_geocoder


In [3]:
import os
import pandas as pd
import reverse_geocoder as rg

# --- 1. Load your CSV data ---
input_file_path = "../content/gdrive/My Drive/Colab_Notebooks/5GCoverageCode/Simplified_Rand_Real.csv"
df = pd.read_csv(input_file_path)

#Get output directory and create if not exist
output_dir = "../content/gdrive/My Drive/Colab_Notebooks/5GCoverageResults/"
os.makedirs(output_dir, exist_ok=True) # Create the output directory if it doesn't exist

# --- 2. Prepare coordinates for batch geocoding ---
coordinates = list(zip(df['SiteLatitude_decDeg'], df['SiteLongitude_decDeg']))

# --- 3. Perform batch reverse geocoding ---
results = rg.search(coordinates)

# --- 4. Extract state information and add to DataFrame ---
states = []
for result in results:
    state = result.get('admin1')  # First try admin1
    if state is None or state == "":
        state = result.get('admin2')  # If admin1 is missing, try admin2
        if state is None or state == "":
            state = result.get('name', 'Not Found')  # If both are missing, use 'name' or 'Not Found'
    states.append(state)

df['State'] = states

# --- 5. Separate into files by state ---
for state in df['State'].unique():
    if state != "Not Found":  # Skip 'Not Found' entries
        state_df = df[df['State'] == state]
        output_file_path = os.path.join(output_dir, f"{state}_sites.csv")
        state_df.to_csv(output_file_path, index=False)
        print(f"Created {output_file_path}")
    else:
        not_found_df = df[df['State'] == "Not Found"]
        output_file_path = os.path.join(output_dir, "sites_not_found.csv")
        not_found_df.to_csv(output_file_path, index=False)
        print(f"Created {output_file_path}")

print(df.head())

FileNotFoundError: [Errno 2] No such file or directory: '../content/gdrive/My Drive/Colab_Notebooks/5G_coverage/Simplified_Rand_Real.csv'